In [9]:
data = """px{a<2006:qkq,m>2090:A,rfg}
pv{a>1716:R,A}
lnx{m>1548:A,A}
rfg{s<537:gd,x>2440:R,A}
qs{s>3448:A,lnx}
qkq{x<1416:A,crn}
crn{x>2662:A,R}
in{s<1351:px,qqz}
qqz{s>2770:qs,m<1801:hdj,R}
gd{a>3333:R,R}
hdj{m>838:A,pv}

{x=787,m=2655,a=1222,s=2876}
{x=1679,m=44,a=2067,s=496}
{x=2036,m=264,a=79,s=2244}
{x=2461,m=1339,a=466,s=291}
{x=2127,m=1623,a=2188,s=1013}
"""
data = open('puzzle.data').read()

from operator import lt, gt

def solve(data):
    workflows_str, parts_str = data.split('\n\n')
    workflows = {}
    for line in workflows_str.splitlines():
        name, steps = line.split('{')
        workflows[name] = []
        steps = steps[:-1].split(',')
        for step in steps[:-1]:
            condition, result = step.split(':')
            op = {'<' : lt, '>' : gt}[condition[1]]
            def process(part, condition=condition, result=result, op=op):
                if op(part[condition[0]], int(condition[2:])):
                    return result
                return False
            workflows[name].append(process)
        def return_result(part, steps=steps):
            return steps[-1]
        workflows[name].append(return_result)

    points = 0
    for part_str in parts_str.splitlines():
        part = {}
        for item in part_str[1:-1].split(','):
            key, value = item.split('=')
            part[key] = int(value)

        workflow_name = 'in'
        while True:
            for operation in workflows[workflow_name]:
                result = operation(part)
                if result:
                    break
            if result == 'A':
                points += sum(part.values())
                break
            if result == 'R':
                break
            workflow_name = result
    
    return points
            
solve(data)

420739

In [10]:
from functools import reduce

def solve2(data: str) -> int:
    workflows_str, parts_str = data.split('\n\n')
    workflows = {}
    for line in workflows_str.splitlines():
        name, steps = line.split('{')
        workflows[name] = steps[:-1].split(',')
    
    open_list = [{l: (1, 4000) for l in 'xmas'}]
    accepted = []
    while open_list:
        part = open_list.pop()
        workflow = 'in'
        while True:
            for instruction in workflows[workflow][:-1]:
                condition, next_step = instruction.split(':')
                attribute, op, value = condition[0], condition[1], int(condition[2:])
                range_ = part[attribute]
                if op == '<':
                    if range_[0] > value:
                        continue
                    if range_[0] < value and range_[1] > value:
                        open_list.append({**part, attribute: (range_[0], value - 1)})
                        part[attribute] = (value, range_[1])
                        continue
                    if range_[1] < value:
                        workflow = next_step
                        break
                elif op == '>':
                    if range_[1] < value:
                        continue
                    if range_[0] < value and range_[1] > value:
                        open_list.append({**part, attribute: (value + 1, range_[1])})
                        part[attribute] = (range_[0], value)
                        continue
                    if range_[0] > value:
                        workflow = next_step
                        break
            else:
                workflow = workflows[workflow][-1]
            
            if workflow == 'A':
                accepted.append(part)
                break
            elif workflow == 'R':
                break
    
    return sum(reduce(lambda a, b: a * (b[1] - b[0] + 1), p.values(), 1) for p in accepted)
            
solve2(data)

130251901420382